In [1]:
import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
import statsmodels.api as sm

C:\Anaconda\lib\site-packages\statsmodels\compat\pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


In [58]:
bwght = pd.read_csv('BWGHT.CSV')

bwght.head(6)

sample =  np.random.permutation(1388)
train = sample[:int(1388/2)]
test = sample[int(1388/2):]
xdata = pd.concat((bwght['cigs'],bwght['faminc']),1).values
xdata = sm.add_constant(xdata)
ydata = bwght['bwght'].values
model = smf.OLS(ydata[train],xdata[train]).fit()
mspe = ((ydata[test]-model.predict(xdata[test]))**2).mean()
mspe

398.91503994985476

In [57]:
sample = np.random.permutation(1388)
xdata = pd.concat((bwght['cigs'],bwght['faminc']),1).values
xdata = sm.add_constant(xdata)
ydata = bwght['bwght'].values
k = 4
mspe = np.zeros(k)
for j in range(k):    
    train_l = sample[:int((3-j)*1388/k)]
    train_u = sample[int((4-j)*1388/k):]
    train = np.concatenate((train_l,train_u),0)
    test = sample[int((3-j)*1388/k):int((4-j)*1388/k)]
    model = smf.OLS(ydata[train],xdata[train]).fit()
    mspe[j] = ((ydata[test]-model.predict(xdata[test]))**2).mean()
mspe

array([ 422.93726797,  401.56619594,  431.60061667,  366.05398075])

In [53]:
np.zeros(4)

array([ 0.,  0.,  0.,  0.])

In [5]:
cd "C:/Users/jap090020/Dropbox/UTDallas/buan6340/post"

C:\Users\jap090020\Dropbox\UTDallas\buan6340\post


In [59]:
import sqlite3

In [67]:
con = sqlite3.connect("bwght.db")
# con = sqlite3.connect(":memory:")
c = con.cursor()


In [63]:
bwght.to_sql('BTABLE',con)

In [65]:
c.execute('SELECT * FROM BTABLE WHERE cigs=50')
print(c.fetchall())

[(1303, 27.5, 2.5, 109.4, 90, None, 12.0, 5, 1, 0, 50, 4.49981, 5.625, 2.5, 3.314186)]


In [68]:
def hailstone(x):
    step = 0
    while x!=1:
        step += 1
        x = x/2 if x%2==0 else 3*x+1
    return step

In [69]:
con.create_function('hs',1,hailstone)
c.execute('SELECT hs(cigs), cigs FROM BTABLE WHERE cigs>30')
for item in c.fetchall():
    print(item)

(8, 40)
(8, 40)
(8, 40)
(8, 40)
(8, 40)
(16, 46)
(8, 40)
(24, 50)


In [70]:
class geomean:
    def __init__(self): # Constructor
        self.sum = 0
        self.count = 0
    def step(self,value): # What to do when we get a new value
        self.sum += np.log(value)
        self.count += 1
    def finalize(self):
        return np.exp(self.sum/self.count)

In [74]:
class geomean2:
    def __init__(self): # Constructor
        self.prod = 1
        self.count = 0
    def step(self,value): # What to do when we get a new value
        self.prod *= value
        self.count += 1
    def finalize(self):
        return (self.prod)**(1/self.count)

In [73]:
con.create_aggregate("GEO",1,geomean)
c.execute('SELECT GEO(bwght),cigs,COUNT(bwght) FROM BTABLE GROUP BY cigs;')
for item in c.fetchall():
    print(item)

(118.12914912512608, 0, 1176)
(122.01151015653141, 1, 3)
(108.25224006111848, 2, 4)
(114.32987300163663, 3, 7)
(108.60626107277508, 4, 9)
(112.90200845344134, 5, 19)
(121.44933131428813, 6, 6)
(119.16422565354631, 7, 4)
(87.69083957636508, 8, 5)
(127.99999999999997, 9, 1)
(110.78738483467163, 10, 55)
(105.91537925146471, 12, 5)
(106.73221579113947, 15, 19)
(108.05121183968193, 20, 62)
(106.91073090784444, 30, 5)
(104.74271003586932, 40, 6)
(121.00000000000003, 46, 1)
(90.0, 50, 1)


In [77]:
con.create_aggregate("GEO2",1,geomean2)
c.execute('SELECT GEO2(bwght),cigs,COUNT(bwght) FROM BTABLE WHERE cigs>0 GROUP BY cigs ;')
for item in c.fetchall():
    print(item)

(122.01151015653136, 1, 3)
(108.25224006111847, 2, 4)
(114.32987300163668, 3, 7)
(108.60626107277498, 4, 9)
(112.90200845344124, 5, 19)
(121.44933131428802, 6, 6)
(119.16422565354631, 7, 4)
(87.69083957636506, 8, 5)
(128.0, 9, 1)
(110.7873848346715, 10, 55)
(105.9153792514647, 12, 5)
(106.73221579113957, 15, 19)
(108.05121183968178, 20, 62)
(106.91073090784451, 30, 5)
(104.74271003586931, 40, 6)
(121.0, 46, 1)
(90.0, 50, 1)


In [66]:
con.commit()
con.close()

In [102]:
xdata = pd.concat((bwght['cigs'],bwght['faminc']),1).values
xdata = sm.add_constant(xdata)
ydata = bwght['bwght'].values
model = smf.OLS(ydata,xdata).fit()
beta = model.params
resid = ydata-model.predict()
beta

array([ 116.97413048,   -0.46340754,    0.09276474])

In [104]:
#beta[1] = 0.0
bstar = np.zeros((10000,3))
for isim in range(10000):
    sample =  np.random.randint(low=0,high=1388,size=(1388,))
    xxdata = xdata[sample]
    yydata = xxdata.dot(beta)+resid[sample]
    model = smf.OLS(yydata,xxdata).fit()
    bstar[isim,:] = model.params
bstar

array([[ 115.75988149,   -0.0403791 ,    0.12711485],
       [ 116.32531678,   -0.07829199,    0.10961686],
       [ 116.1047558 ,    0.13365071,    0.08335042],
       ..., 
       [ 117.05160219,    0.05145657,    0.08013914],
       [ 118.28630532,   -0.14770299,    0.05120206],
       [ 117.99727985,    0.03185845,    0.06469627]])

In [105]:
np.set_printoptions(suppress=True)
bstar[:,0].sort()
bstar[:,1].sort()
bstar[:,2].sort()
print(bstar[int(10000*0.025)])
print(bstar[int(10000*0.975)])

[ 114.96070174   -0.17746492    0.03597279]
[ 118.9930044     0.16916835    0.14896821]


In [118]:
xdata = pd.concat((bwght['cigs'],bwght['faminc']),1).values
xdata = sm.add_constant(xdata)
ydata = bwght['bwght'].values
model = smf.OLS(ydata,xdata).fit()
beta = model.params
resid = ydata-model.predict()
model.tvalues

array([ 111.51182089,   -5.06031465,    3.17819498])

In [114]:
beta = np.zeros((3,))
tstar = np.zeros((10000,3))
for isim in range(10000):
    sample =  np.random.randint(low=0,high=1388,size=(1388,))
    xxdata = xdata[sample]
    yydata = xxdata.dot(beta)+resid[sample]
    model = smf.OLS(yydata,xxdata).fit()
    tstar[isim,:] = model.tvalues
tstar[:,0].sort()
tstar[:,1].sort()
tstar[:,2].sort()
print(tstar[int(10000*0.025)])
print(tstar[int(10000*0.975)])

[-1.94493614 -1.91895985 -1.91704485]
[ 1.92886679  1.90037637  1.89645557]


[-1.9272547  -1.89883774 -1.90314497]
[ 1.9493734   1.86575141  1.9077566 ]
